Consider a sky-diver falling towards the surface of Earth. The diver brought with them a temperature sensor that took multiple measurements along the descent. The diver is interested in determining their terminal velocity given these measurements.

Problem: Given a temperature measurement, can one estimate the velocity as a function of time?

In [1]:
using LinearAlgebra
using Random
using PlotlyJS

include("kalmanflt.jl");

WebIO._IJuliaInit()

The system is a particle falling due to gravity WITH air resistance. For this simulation, the falling object will acheive terminal velocity. The velocity as a function of time is modeled as

$$
v(t) = v_{\infty} \tanh\left(\frac{g t}{v_{\infty}}\right)
$$

where $v_{\infty}$ is the terminal velocity as time approaches infinity and $g$ is the acceleration due to gravity. We want to model the position and velocity of an object falling, but we CANNOT use the following equations

$$
\begin{align}
h &= h_{0} + v_0 t + \frac{1}{2} a t^{2} \\
v &= v_{0} + g t
\end{align}
$$

since the acceleration is NOT constant in time

$$
\begin{align}
a &= \frac{dv}{dt} = g \ \text{sech}^2\left(\frac{g t}{v_{\infty}}\right)
\end{align}
$$

Let's consider the following model instead

$$
\begin{align}
h \rightarrow h + \delta h &= \sum\limits_{n=0}^{\infty}\left.\left(\frac{d}{dt}\right)^{n} h\right|_{t=T}\frac{\left(\Delta t\right)^n}{n!} \\
     &= h(T) + v(T) \Delta t + \frac{1}{2} a(T) \left(\Delta t\right)^2 + \frac{1}{3!} j(T) \left(\Delta t\right)^3 + \frac{1}{4!} s(T) \left(\Delta t\right)^4 + \mathcal{O}(t^5)
\end{align}
$$

The terms in the sum above are given by

$$
\begin{align}
v(t) &= v_{\infty} \tanh\left(\frac{g t}{v_{\infty}}\right) & \text{Velocity}\\
a(t) &= \frac{dv}{dt}     = g \ \text{sech}^2\left(\frac{g t}{v_{\infty}}\right) & \text{Acceleration}\\
j(t) &= \frac{d^2v}{dt^2} = -\frac{2 g^2}{v_{\infty}} \tanh \left(\frac{g t}{v_{\infty}}\right) \text{sech}^2\left(\frac{g t}{v_{\infty}}\right) & \text{Jerk} \\
s(t) &= \frac{d^3v}{dt^3} = \frac{2 g^3}{v_{\infty}^2} \left(\cosh \left(\frac{2 g t}{v_{\infty}}\right)-2\right) \text{sech}^4\left(\frac{g t}{v_{\infty}}\right) & \text{Snap} \\
\vdots
\end{align}
$$

and we must determine when to truncate the series. Let's plot each derivative.

In [2]:
accel_ms2 = -9.8 # meter / sec^2
v_inf = 54 # meter / sec
time_steps = 100
delta_time_sec = 0.25  # sec
time_axis_sec = [delta_time_sec * (t-1) for t in 1:time_steps];

In [3]:
plot(
    [
        # Velocity
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[v_inf * tanh(accel_ms2 * t / v_inf) for t in time_axis_sec],
            name="Velocity"
        ),
        # Acceleration
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[accel_ms2 * sech(accel_ms2 * t / v_inf)^2 for t in time_axis_sec],
            name="Acceleration"
        ),
        # Jerk
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[-2 * accel_ms2^2 / v_inf * tanh(accel_ms2 * t / v_inf)^2 * sech(accel_ms2 * t / v_inf)^2 for t in time_axis_sec],
            name="Jerk"
        ),
        # Snap
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[2 * accel_ms2^3 / v_inf^2 * (cosh(2 * accel_ms2 * t / v_inf) - 2) * sech(accel_ms2 * t / v_inf)^4 for t in time_axis_sec],
            name="Snap"
        ),
    ],
    Layout(
        title="Velocity Model",
        xaxis_title="Time (second)",
        yaxis_title="(d/dt)^n v per Unit Time",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"

For this example, we will truncate the series up to the jerk term.

What we need now is a mapping between temperature and altitude.
I will use the data from https://www.engineeringtoolbox.com/air-altitude-temperature-d_461.html and linearize the temperature model. For the terminal velocity, I make-up a weak, linear relationship between temperature and terminal velocity.

In [11]:
function get_velocity_infinity_ms(
        temperature_K::Float64
    )::Float64
    """Get the velocity as time goes to infinity.
    v_inf between ~46 and ~54 meters / sec

    :param temperature_K: Temperature in Kelvin
    :return: Terminal velocity in meters / sec
    """
    return -0.1481 * temperature_K + 94.44
end

function get_temperature_K(
        altitude_m::Float64, 
        vary_slope_by::Float64=0.0, 
        vary_intercept_by::Float64=0.0
    )::Float64
    """Get the temperature in Kelvin as a function of altitude

    :param altitude_m: Altitude in meters
    :param vary_slope_by: Positive definite constant to vary the slope of temperature changes
    :param vary_intercept_by: Positive definite constant to vary the intercept of temperature changes
    :returns: Float64
    """
    slope = -0.006_835 # Kelvin / meter
    intercept = 288.706 # Kelvin
    if vary_slope_by > 0
        slope *= vary_slope_by
    end
    if vary_intercept_by > 0
        intercept *= vary_intercept_by
    end
    temerature = slope * altitude_m + intercept
    return temerature
end;

In [9]:
temperature_range_K = 273.0:300.0

plot(
    [
        scatter(
            mode="markers+lines",
            x=temperature_range_K,
            y=[get_velocity_infinity_ms(T) for T in temperature_range_K]
        )
    ],
    Layout(
        title="Terminal Velocity Model",
        xaxis_title="Temperature (Kelvin)",
        yaxis_title="Velocity (meters / seconds)",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"

In [14]:
altitude_range_m = 0.0:100.0:3000.0

plot(
    [
        scatter(
            mode="markers+lines",
            x=altitude_range_m,
            y=[get_temperature_K(A) for A in altitude_range_m]
        )
    ],
    Layout(
        title="Temperature Model",
        xaxis_title="Altitude (meters)",
        yaxis_title="Velocity (meters / seconds)",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"

With those two models established, let's define the observation matrix as

$$
\begin{align}
H &= H(t) \\
&=
\begin{bmatrix}
\frac{T(a)}{a} & 0
\end{bmatrix}
\end{align}
$$

where $a$ is the altitude and $T$ is the temperature as a function of a

In [19]:
function get_observation_mat(
        altitude_m::Float64,
        vary_temperature_slope_by::Float64=0.0,
        vary_temperature_intercept_by::Float64=0.0
    )::Matrix{Float64}
    """Get the observation matrix
    
    :param altitude_m: Altitude in meters
    :param vary_temperature_slope_by: Positive definite constant to vary the slope of temperature changes
    :param vary_temperature_intercept_by: Positive definite constant to vary the intercept of temperature changes 
    :returns: Matrix
    """
    temperature_K = get_temperature(altitude_m, 
                                  vary_temperature_slope_by, 
                                  vary_temperature_intercept_by)
    observation_mat = [temperature_K/altitude_m 0.0]
    return observation_mat
end;

Let's consider the state transition and control matrices. They are given by
$$
\begin{align}
x_n =& F x_{n-1} + G u \\
    =& 
\begin{bmatrix}
1 & \Delta t \\
0 & 1
\end{bmatrix}
\begin{bmatrix}
h_{n-1} \\
v_{n-1}
\end{bmatrix}
+
\begin{bmatrix}
\frac{1}{2} a(T) \left(\Delta t\right)^2 + \frac{1}{3!} j(T) \left(\Delta t\right)^3 & 0 \\
0 & a(T) \Delta t + \frac{1}{2} j(T) \left(\Delta t\right)^2
\end{bmatrix}
\begin{bmatrix}
1 \\
1
\end{bmatrix} \\
\begin{bmatrix}
h_n \\
v_n
\end{bmatrix}
=& 
\begin{bmatrix}
h(t_{n-1}) + v(t_{n-1}) \Delta t + \frac{1}{2} a(t_n) \left(\Delta t\right)^2 + \frac{1}{3!} j(t_{n-1}) \left(\Delta t\right)^3 \\
v(t_{n-1}) + a(t_{n-1}) \Delta t + \frac{1}{2} j(t_{n-1}) \left(\Delta t\right)^2
\end{bmatrix}
\end{align}
$$

In [18]:
function get_control_mat(
        state_vec::Vector,
        total_time_sec::Float64,
        delta_time_sec::Float64
    )::Matrix{Float64}
    """Get the control matrix
    
    :param state_vec: State vector
    :param total_time_sec: Total time of system measurements
    :param delta_time_sec: Time differential between measurements
    :returns: Matrix
    """

    accel_ms2 = -9.8 # meter / sec^2
    temperature_K = get_temperature(state_vec[1])
    vel_inf_ms = get_velocity_infinity(temperature_K)
    arg = accel_ms2 * total_time_sec / vel_inf_ms
    current_acceleration_ms2 = accel_ms2 * sech(arg)^2
    current_jerk_ms3 = -2 * accel_ms2^2 / vel_inf_ms * sech(arg)^2 * tanh(arg)
    control_mat11 = (1. / factorial(2) * current_acceleration_ms2 * delta_time_sec^2
                     + 1. / factorial(3) * current_jerk_ms3 * delta_time_sec^3)
    control_mat_12 = 0.
    control_mat_21 = 0.
    control_mat_22 = -(current_acceleration_ms2 * delta_time_sec
                       + 1. / factorial(2) * current_jerk_ms3 * delta_time_sec^2)
    control_mat = [control_mat11 control_mat_12; control_mat_21 control_mat_22]
    return control_mat
end;

In [22]:
function get_model_estimate(
        state_vec::Vector,
        total_time_sec::Float64,
        delta_time_sec::Float64
    )::Vector{Float64}
    """Get the model estimate for current state

    :param state_vec: State vector
    :param total_time_sec: Total time of system measurements
    :param delta_time_sec: Time differential between measurements
    :returns: Vector
    """

    state_transition = [1.0 -delta_time_sec; 0.0 1.0]
    control_mat = get_control_mat(state_vec, total_time_sec, delta_time_sec)
    external_input_vector = Vector([1, 1])
    model_estimate_vec = (state_transition * state_vec
                          + control_mat * external_input_vector)
    return model_estimate_vec
end

function get_measurement_vec(
        true_state::Vector,
        vary_temperature_slope_by::Float64=0.0,
        vary_temperature_intercept_by::Float64=0.0
    )::Vector{Float64}
    """Get the simulated measurement as a function of the true state

    :param true_state: True state vector
    :param vary_temperature_slope_by: Positive definite constant to vary the slope of temperature changes
    :param vary_temperature_intercept_by: Positive definite constant to vary the intercept of temperature changes 
    :returns: Measurement vector
    """
    """Get the next measurement
    In a 'real' physical system, we would not need to simulate the measurement"""

    altitude_m = true_state[1]
    observation_mat = get_observation_mat(altitude_m, 
                                          vary_temperature_slope_by, 
                                          vary_temperature_intercept_by)

    artificial_noise_K = 0.1 * randn(1) # Kelvin
    measurement_vec::Vector = observation_mat * true_state + artificial_noise_K
    return measurement_vec

end;